# Install and load necesary packages

In [3]:
# Please don't change this cell

import pandas as pd
import numpy as np  

import warnings
warnings.filterwarnings("ignore")

## Load the dataset using pandas

In [5]:
# Please don't change this cell
df = pd.read_csv('ml-100k/u.data', names=['user_id', 'item_id', 'rating', 'timestamp'], sep='\t')

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Split dataset


## Randomly select one rating from each user as test set

In [8]:
# please do not change this cell

from sklearn.model_selection import train_test_split

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(str(n_users) + ' users')
print(str(n_items) + ' items')

train_df, test_df = train_test_split(df, test_size=0.2, random_state = 10)
train_df, test_df

# Training Dataset
train_ds = np.zeros((n_users, n_items))
item_popularity = np.zeros(n_items)
for row in train_df.itertuples():
    train_ds[row[1]-1, row[2]-1] = row[3]
    item_popularity[row[2]-1] =  item_popularity[row[2]-1] + 1
#train_ds = pd.DataFrame(train_ds)

# Testing Dataset
testsize = 0
test_ds = np.zeros((n_users, n_items))
for row in test_df.itertuples():
    if item_popularity[row[2]-1] > 30:
        test_ds[row[1]-1, row[2]-1] = row[3]
        testsize = testsize + 1
#test_ds = pd.DataFrame(test_ds)

print("Construct the rating matrix based on train_df:")
print(train_ds)

print("Construct the rating matrix based on test_df:")
print(test_ds)

print("Testsize = " + str(testsize))

943 users
1682 items
Construct the rating matrix based on train_df:
[[0. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
Construct the rating matrix based on test_df:
[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Testsize = 17678


# MAE and RMSE Utils

In [10]:
# Please don't change this cell
# you can use this devaluate Utils here, and you can also implement your own MAE and RMSE calculation. 

EPSILON = 1e-9

def evaluate(test_ds, predicted_ds):
    '''
    Function for evaluating on MAE and RMSE
    '''
    # MAE
    mask_test_ds = test_ds > 0
    MAE = np.sum(np.abs(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32))

    # RMSE
    RMSE = np.sqrt(np.sum(np.square(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32)))

    return MAE, RMSE

# Your Solution for Method 1

In [12]:
# Write your code here for Method 1
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

# Create a matrix full of 0 to put value in it
usersNum = np.zeros((n_items, n_users))
for row in df.itertuples():
     # loops through each row in your df
    # if there's rating in df, then mark 1 in usersNum matrix
    usersNum[row[2]-1, row[1]-1] = 1
# Get row the sum value for further mean calculation
user_rating_count = usersNum.sum(axis=0)
# print(user_rating_count)

In [13]:
user_ratings = np.zeros((n_items, n_users))
for row in df.itertuples():
     # same, but this time put the actule value in the matrix
    user_ratings[row[2]-1, row[1]-1] = row[3]

user_rating_sum = user_ratings.sum(axis=0)
#calculate user average, add EPSILON in case user_rating_count is 0
user_avg = user_rating_sum/(user_rating_count + EPSILON)
# print(user_avg)

In [14]:
# create another matrix
predicted_ds_solution1 = np.zeros((n_items, n_users))

for i in range(n_items):
    for j in range(n_users):
        # put the all useraverage in the matrix as a simple prediction
        predicted_ds_solution1[i][j] = user_avg[j]

# transpose it to match test_ds shape to evaluate MAE and RMSE
predicted_ds_solution1 = predicted_ds_solution1.transpose()
# print(predicted_ds_solution1)

In [15]:
MAE_solution1, RMSE_solution1 = evaluate(test_ds, predicted_ds_solution1)

In [16]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution1, RMSE_solution1))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.8172422827915611, RMSE: 1.0189153668434672


# Your Solution for Method 2

In [18]:
# Write your code here for Method 2
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

# Same as method 1 but this time use (n_users, n_items) to create matrix
# No need to transpose to match test_ds shape at the end
ratingsNum = np.zeros((n_users, n_items))
for row in df.itertuples():
     # loops through each row in your df
    ratingsNum[row[1]-1, row[2]-1] = 1
item_rating_count = ratingsNum.sum(axis=0)
# print(item_rating_count)

In [19]:
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3] # rating

item_rating_sum = ratings.sum(axis=0)# from up to down, row wise sum
item_avg = item_rating_sum/(item_rating_count + EPSILON)
# print(item_avg)

In [20]:
predicted_ds_solution2 = np.zeros((n_users, n_items))

for i in range(n_users):
    for j in range(n_items):
        predicted_ds_solution2[i][j] = item_avg[j]
# print(predicted_ds_solution2)

In [21]:
MAE_solution2, RMSE_solution2 = evaluate(test_ds, predicted_ds_solution2)


In [22]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution2, RMSE_solution2))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7893782727783716, RMSE: 0.9921327331945785


# Your Solution for Method 3

### Create matrix to hold Pearson correlation values between users

In [25]:
# Write your code here for Method 3
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

# GAMMA = 0

# Computes similarity between users	
# Create an empty matrix to hold Pearson correlation values between users
np_user_pearson_corr = np.zeros((n_users, n_users))  # Shape: (num_users, num_users)

for i, user_i_vec in enumerate(train_ds):
    # enumerate, On the first loop: i = 0, user_i_vec = [5, 3, 0, 1]
    for j, user_j_vec in enumerate(train_ds):
        
        # ratings corated by the current pair of users
        # True where user i is rated
        mask_i = user_i_vec > 0 
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        
        if len(corrated_index) == 0:
            continue # Skip if no co-rated items

       
        # np.clip turns non-zero val into 1,keeps 0 as 0
        # e.g.
        # user_i_vec = [5, 3, 0, 1]
        # np.clip(user_i_vec, 0, 1) = [1, 1, 0, 1]

        
        i_num = np.sum(np.clip(user_i_vec, 0, 1))
        j_num = np.sum(np.clip(user_j_vec, 0, 1))

        
        # Average value of user_i_vec and user_j_vec    
        mean_user_i = np.sum(user_i_vec) / (i_num + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (j_num + EPSILON)
        
        

        # compute pearson corr
        # Subtract the mean from ratings (normalizing)
        # Normalize because user might tend to give high or low rating
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j
        
        # Dot product: a way to measure similarity between two vectors.
        # sim = dot(a, b) / (||a|| * ||b||)
        # a = [2, -2] ; b = [1.5, -1.5]
        # dot = (2*1.5) + (-2*-1.5) = 3 + 3 = 6 the higher the better
        # ||a|| = sqrt(2^2 + (-2)^2) = sqrt(8)
        # ||b|| = sqrt(1.5^2 + (-1.5)^2) = sqrt(4.5)
        # similarity = 6 / (sqrt(8) * sqrt(4.5)) ≈ 1.0 → perfect similarity

        # Pearson similarity formula
        # dot product / (magnitude × magnitude) 
        # similarity = dot_product / (length_of_vector_1 × length_of_vector_2)
        # → (+1)best +correlation, (-1)best -correlation, (0)no correlation
        
        # Compute squared terms and their square roots for the denominator
        # TNormalisation part in correlation. 
        # Scales the result so it's always between -1 and 1.
        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))
        #  Think length = sqrt(x² + y² + ...)
        
        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # Optional significance weighting
        weighted_sim = sim  # You could apply weighting here
        #weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim
        
        
        # Store result in similarity matrix
        np_user_pearson_corr[i][j] = weighted_sim

        
        
# print(np_user_pearson_corr)

#np_user_pearson_corr = np.array([
#     [1.00, 0.80, 0.90],  # user 0’s similarity with others
#     [0.80, 1.00, 0.60],
#     [0.90, 0.60, 1.00]
# ])
# 1 is the best, -1 is the worst

In [26]:

#Check the number of reviews to fine tune K
# K can't be too big or it'll include too much noise
# print(df.shape)


K = 456
EPSILON = 1e-9


def User_based_prediction_generator(K_user):
    np_predictions = np.zeros((n_users, n_items))


    for (i, j), rating in np.ndenumerate(test_ds):
        # what's np.ndenumerate? 
        #     arr = np.array([ [1, 0],
        #                      [0, 2]])
        #     for (i, j), value in np.ndenumerate(arr):
        #         print('row:', i,'col:',  j,'val:', value)
        #     row: 0 col: 0 val: 1
        #     row: 0 col: 1 val: 0
        #     row: 1 col: 0 val: 0
        #     row: 1 col: 1 val: 2
    
        # Loop through each (user i, item j) in test_ds
        if rating > 0:
            # We only predict for items that appear in test set
            # print(str(i) + " " + str(j))
            
            # find top-k most similar users as the current user, remove itself
            # the index_num of smallest to biggest ....
            # np_lst = [1.0, 0.3, 0.7] 
            # if want to get (0.3, 0.7, 1.0) in ascending order,
            # the index is np.argsort(np_lst) = [1, 2, 0]
            asc_index_num = np.argsort(np_user_pearson_corr[i])
            sim_user_ids = asc_index_num[-(K + 1):-1]
            ### useful link for how to access values in numpy arrays: https://numpy.org/doc/stable/reference/arrays.indexing.html
            
            # the coefficient values of similar users
            sim_val = np_user_pearson_corr[i][sim_user_ids]
            # the average value of the current user's ratings
            sim_users = train_ds[sim_user_ids]
            
            num_of_train_ds = np.sum(np.clip(train_ds[i], 0, 1))
            # np.clip(..., 0, 1) turns ratings into 1/0 to count how many rated
            num_of_sum_byusers = np.sum(np.clip(sim_users, 0, 1), axis=1)
            # axis = 1 meand sum by rows(each users)
    
            
            user_mean = np.sum(train_ds[i]) / ( num_of_train_ds + EPSILON)
            sim_user_mean = np.sum(sim_users, axis=1) / ( num_of_sum_byusers+ EPSILON)
            
            # select the users who rated item j
            mask_rated_j = sim_users[:, j] > 0
    
            # weighted_diffs
            # sim(u, v) * (r_vj - mean_v)
            sim_r_sum_mean =  (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j]) * sim_val[mask_rated_j]
            # For each neighbor who rated item j:
            # take their rating
            # subtract their average (to normalize it)
            # multiply by their similarity with the current user
            # We trust: How much above or below the avg they rated — and how much we trust (similarity).
    
            # Prediction Formula
            # prediction = user_mean + sum(weighted_diffs) / sum(similarities only if they rated this item)
            np_predictions[i][j] = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
            # Ensures: Prediction stays in valid rating range: [0, 5]
            np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
    return np_predictions

predicted_ds_solution3 = User_based_prediction_generator(K)
# print(predicted_ds_solution3)

# train_ds = np.array([ # train rating on item 0,1,3 (we try to predict 2)
#     [5, 3, 0, 1],  # user 0
#     [4, 0, 0, 1],  # user 1
#     [1, 1, 0, 5],  # user 2
# ])
# test_ds = np.array([
#     [0, 0, 4, 0],  # user 0’s test rating on item 2
#     [0, 0, 3, 0],
#     [0, 0, 2, 0],
# ])


In [27]:
MAE_solution3, RMSE_solution3 = evaluate(test_ds, predicted_ds_solution3)


In [28]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution3, RMSE_solution3))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7309371979267135, RMSE: 0.9367076992199926


# Your Solution for Method 4

In [30]:
# Write your code here for Method 4
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

DELTA = 25
EPSILON = 1e-9

# Similar to method 3 but this time use (n_items, n_items)
np_item_pearson_corr = np.zeros((n_items, n_items))
# 	Computes similarity between items(transpose)
for i, item_i_vec in enumerate(train_ds.T):
    for j, item_j_vec in enumerate(train_ds.T):

        # ratings corated by the current pair od items
        mask_i = item_i_vec > 0
        mask_j = item_j_vec > 0

        # corrated index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of item_i_vec and item_j_vec
        mean_item_i = np.sum(item_i_vec) / (np.sum(np.clip(item_i_vec, 0, 1)) + EPSILON)
        mean_item_j = np.sum(item_j_vec) / (np.sum(np.clip(item_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        item_i_sub_mean = item_i_vec[corrated_index] - mean_item_i
        item_j_sub_mean = item_j_vec[corrated_index] - mean_item_j

        r_ui_sub_ri_sq = np.square(item_i_sub_mean)
        r_uj_sub_rj_sq = np.square(item_j_sub_mean)

        r_ui_sub_ri_sq_sum_sqrt = np.sqrt(np.sum(r_ui_sub_ri_sq))
        r_uj_sub_rj_sq_sum_sqrt = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(item_i_sub_mean * item_j_sub_mean) / (r_ui_sub_ri_sq_sum_sqrt * r_uj_sub_rj_sq_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), DELTA) / DELTA) * sim

        np_item_pearson_corr[i][j] = weighted_sim

# print(np_item_pearson_corr)

In [31]:

K = 456
EPSILON = 1e-9

def Item_based_prediction_generator(K_item):
    
    np_predictions = np.zeros((n_users, n_items))
    
    for (i, j), rating in np.ndenumerate(test_ds):
        if rating > 0:
            # find top-k most similar items as the current item, remove itself
            sim_item_ids = np.argsort(np_item_pearson_corr[j])[-(K + 1):-1]
    
            # the coefficient values of similar items
            sim_val = np_item_pearson_corr[j][sim_item_ids]
    
            # the average value of the current item's ratings
            sim_items = train_ds.T[sim_item_ids]
            item_mean = np.sum(train_ds.T[j]) / (np.sum(np.clip(train_ds.T[j], 0, 1)) + EPSILON)
            sim_item_mean = np.sum(sim_items, axis=1) / (np.sum(np.clip(sim_items, 0, 1), axis=1) + EPSILON)
    
            # sim(u, v) * (r_v - mean_v)
            sim_r_sum_mean = sim_val * (sim_items[:, i] - sim_item_mean) 
    
            # filter unrated items
            w = np.clip(sim_items[:, i], 0, 1)
            sim_r_sum_mean *= w
            sim_val *= w
    
            if np.sum(sim_val) == 0:
                np_predictions[i][j] = item_mean
            else:
                np_predictions[i][j] = item_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val) + EPSILON)
                
            np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
    return np_predictions
        

predicted_ds_solution4 = Item_based_prediction_generator(K)

# print(predicted_ds_solution4)

In [32]:
MAE_solution4, RMSE_solution4 = evaluate(test_ds, predicted_ds_solution4)

In [33]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution4, RMSE_solution4))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7120312529582123, RMSE: 0.910277895204705


# Your Solution for Method 5

In [35]:
# Write your code here for Method 5
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

lam = 0.046
user_pred = predicted_ds_solution3
item_pred = predicted_ds_solution4
    
hybrid_pred = (lam * user_pred) + ((1 - lam) * item_pred)
# print(min(df['rating']))
# print(max(df['rating']))
# Find the rating range
hybrid_pred = np.clip(hybrid_pred, 0, 5)
# prediction range from 0 to 5

MAE_solution5, RMSE_solution5  = evaluate(test_ds, hybrid_pred)



In [36]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution5, RMSE_solution5))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7119513883801925, RMSE: 0.9102402362379636
